In [1]:
import numpy as np
import pandas as pd
from my_stuff import MufexKeys
from quantfreedom.exchanges.mufex_exchange.mufex import Mufex
from quantfreedom.enums import DynamicOrderSettingsArrays
from quantfreedom.enums import *
from quantfreedom.helper_funcs import dos_cart_product
from quantfreedom.strategies.strategy import (
    strat_bt_create_ind,
    strat_evaluate,
    strat_get_current_ind_settings,
    strat_get_ind_set_str,
    strat_get_total_ind_settings,
)
from quantfreedom.simulate import sim_df_backtest
from quantfreedom.sim_order_records import sim_get_or
import requests

mufex_main = Mufex(
    api_key=MufexKeys.api_key,
    secret_key=MufexKeys.secret_key,
    use_test_net=False,
)

%load_ext autoreload
%autoreload 2

In [2]:
candles_np = mufex_main.get_candles(symbol="BTCUSDT", timeframe="5m", candles_to_dl=1000)

It took 00 mins and 00 seconds to download 1000 candles


In [3]:
mufex_main.set_exchange_settings(
    symbol="BTCUSDT",
    position_mode=PositionModeType.HedgeMode,
    leverage_mode=LeverageModeType.Isolated,
)
backtest_settings = BacktestSettings(
    divide_records_array_size_by=1,
    gains_pct_filter=-90,
    total_trade_filter=60,
    upside_filter=-np.inf,
)
dos_arrays = DynamicOrderSettingsArrays(
    entry_size_asset=np.array([0]),
    max_equity_risk_pct=np.array([9]),
    max_trades=np.array([5]),
    num_candles=np.array([100]),
    risk_account_pct_size=np.array([3]),
    risk_reward=np.array([2]),
    sl_based_on_add_pct=np.array([0.01]),
    sl_based_on_lookback=np.array([20]),
    sl_bcb_type=np.array([CandleBodyType.Low]),
    sl_to_be_cb_type=np.array([CandleBodyType.Nothing]),
    sl_to_be_when_pct=np.array([0]),
    sl_to_be_ze_type=np.array([0]),
    static_leverage=np.array([0]),
    trail_sl_bcb_type=np.array([CandleBodyType.Low]),
    trail_sl_by_pct=np.array([0.5]),
    trail_sl_when_pct=np.array([1]),
)
static_os = StaticOrderSettings(
    increase_position_type=IncreasePositionType.RiskPctAccountEntrySize,
    leverage_strategy_type=LeverageStrategyType.Dynamic,
    long_or_short=LongOrShortType.Long,
    logger_bool=True,
    pg_min_max_sl_bcb=PriceGetterType.Min,
    sl_to_be_bool=False,
    z_or_e_type=ZeroOrEntryType.Nothing,
    sl_strategy_type=StopLossStrategyType.SLBasedOnCandleBody,
    tp_strategy_type=TakeProfitStrategyType.RiskReward,
    tp_fee_type=TakeProfitFeeType.Limit,
    trail_sl_bool=True,
)
dos_cart_arrays = dos_cart_product(
    dos_arrays=dos_arrays,
)

ExchangeSettings(
    limit_fee_pct = 0.0003,
    max_leverage = 150.0,
    market_fee_pct = 0.0009,
    mmr_pct = 0.004,
    min_leverage = 1.0,
    max_asset_size = 100.0,
    min_asset_size = 0.001,
    asset_tick_step = 3,
    position_mode = 1,
    leverage_mode = 1,
    price_tick_step = 1,
    leverage_tick_step = 2,
)


In [4]:
sim_df_backtest(
    backtest_settings=backtest_settings,
    candles=candles_np,
    dos_cart_arrays=dos_cart_arrays,
    evaluate=strat_evaluate,
    exchange_settings=mufex_main.exchange_settings,
    get_current_ind_settings=strat_get_current_ind_settings,
    get_ind_set_str=strat_get_ind_set_str,
    get_total_ind_settings=strat_get_total_ind_settings,
    ind_creator=strat_bt_create_ind,
    logger_type=LoggerType.File,
    starting_equity=1000.0,
    static_os=static_os,
)

Starting the backtest now ... and also here are some stats for your backtest.

Total indicator settings to test: 6
Total order settings to test: 1
Total combinations of settings to test: 6
Total candles: 1,000
Total candles to test: 6,000


,ind_set_idx,dos_index,total_trades,wins,losses,gains_pct,win_rate,to_the_upside,fees_paid,total_pnl,ending_eq
0,3,0,70.0,11,59,-85.72,15.71,0.0,805.737,-857.1645,142.8355


In [10]:
order_records_df = sim_get_or(
    backtest_settings=backtest_settings,
    candles=candles_np,
    dos_cart_arrays=dos_cart_arrays,
    evaluate=strat_evaluate,
    exchange_settings=mufex_main.exchange_settings,
    get_current_ind_settings=strat_get_current_ind_settings,
    get_ind_set_str=strat_get_ind_set_str,
    get_total_ind_settings=strat_get_total_ind_settings,
    ind_creator=strat_bt_create_ind,
    logger_type=LoggerType.File,
    starting_equity=1000.0,
    static_os=static_os,
    ind_set_index=0,
    dos_index=0,
)

In [6]:
order_records_df

,ind_set_idx,or_set_idx,bar_idx,timestamp,datetime,order_status,equity,available_balance,cash_borrowed,cash_used,...,entry_size_usd,entry_price,exit_price,position_size_asset,position_size_usd,realized_pnl,sl_pct,sl_price,tp_pct,tp_price
0,0,0,102,1698141600000,2023-10-24 10:00:00,EntryFilled,1000.0000,949.8100,2461.590,50.190,...,2511.780,34345.0,NaN,0.073,2511.780,NaN,1.02,33996.3,2.5,35206.9
1,0,0,105,1698142500000,2023-10-24 10:15:00,EntryFilled,1000.0000,882.4550,6102.409,117.545,...,3758.364,34297.9,NaN,0.183,6270.144,NaN,0.81,34049.9,2.0,35025.7
2,0,0,106,1698142800000,2023-10-24 10:20:00,EntryFilled,1000.0000,830.7600,8823.741,169.240,...,2890.572,34321.9,NaN,0.267,9160.716,NaN,0.80,34049.9,2.1,35041.9
3,0,0,134,1698151200000,2023-10-24 12:40:00,MovedTSL,1000.0000,830.7600,8823.741,169.240,...,2890.572,34321.9,NaN,0.267,9160.716,NaN,0.63,34541.5,2.1,35041.9
4,0,0,138,1698152400000,2023-10-24 13:00:00,StopLossFilled,1040.9896,1040.9896,NaN,NaN,...,NaN,NaN,34541.5,NaN,NaN,40.9896,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,0,0,870,1698372000000,2023-10-27 02:00:00,EntryFilled,469.5763,377.9000,7801.841,91.677,...,3545.257,33846.7,NaN,0.235,7944.283,NaN,0.16,33773.9,0.8,34107.8
123,0,0,871,1698372300000,2023-10-27 02:05:00,EntryFilled,469.5763,344.4760,10491.780,125.101,...,2815.040,33891.7,NaN,0.318,10759.323,NaN,0.20,33773.9,0.9,34146.6
124,0,0,876,1698373800000,2023-10-27 02:30:00,TakeProfitFilled,553.5914,553.5914,NaN,NaN,...,NaN,NaN,34146.6,NaN,NaN,84.0151,NaN,NaN,NaN,NaN
125,0,0,877,1698374100000,2023-10-27 02:35:00,EntryFilled,553.5914,524.7660,1442.077,28.825,...,1470.902,34107.0,NaN,0.043,1470.902,NaN,0.98,33773.9,2.4,34936.6
